# 📝 Text Summarization — BART Fine-tuning on SAMSum

This notebook trains **facebook/bart-large-cnn** on the **SAMSum** dialogue summarization dataset.

**Requirements:**
- Kaggle: Enable **GPU T4 x2** under Settings → Accelerator
- Colab: Runtime → Change runtime type → **T4 GPU**

**Training time:** ~2 hours on T4 GPU (3 epochs)

## 1. Install Dependencies

In [3]:
!pip install -q transformers==4.36.2 datasets==2.16.1 evaluate==0.4.1 \
    rouge-score==0.1.2 accelerate==0.25.0 sentencepiece protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 58.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 18.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_co

## 2. Verify GPU Availability

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
   
else:
    print("⚠️ No GPU detected! Training will be very slow. Enable GPU in notebook settings.")

PyTorch version: 2.10.0+cu128
CUDA available: True
GPU: Tesla T4


## 3. Load SAMSum Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("samsum")
print(f"Train: {len(dataset['train'])} examples")
print(f"Validation: {len(dataset['validation'])} examples")
print(f"Test: {len(dataset['test'])} examples")
print(f"\nColumns: {dataset['train'].column_names}")
print(f"\n--- Example ---")
print(f"Dialogue: {dataset['train'][0]['dialogue'][:200]}...")
print(f"Summary: {dataset['train'][0]['summary']}")

## 4. Load Tokenizer & Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "facebook/bart-large-cnn"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

print(f"Model parameters: {model.num_parameters() / 1e6:.0f}M")
print(f"Vocab size: {tokenizer.vocab_size}")

## 5. Tokenize Dataset

In [ ]:
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128

def preprocess_function(examples):
    """Tokenize dialogues and summaries."""
    inputs = examples["dialogue"]
    targets = examples["summary"]
    
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
    desc="Tokenizing"
)

print(f"Tokenized train: {len(tokenized_dataset['train'])}")
print(f"Tokenized test: {len(tokenized_dataset['test'])}")

## 6. Training Configuration

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import evaluate

# Load ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Compute ROUGE metrics during evaluation."""
    predictions, labels = eval_pred
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels (padding)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {k: round(v, 4) for k, v in result.items()}


OUTPUT_DIR = "./bart-samsum-finetuned"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    
    # Evaluation
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    # Performance
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    
    # Logging
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_steps=100,
    report_to="none",
    
    # Generation for eval metrics
    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LENGTH,
)

print("Training configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  FP16: {training_args.fp16}")

## 7. Train the Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

print("Starting training...")
train_result = trainer.train()

print(f"\n{'='*50}")
print("Training Complete!")
print(f"{'='*50}")
print(f"Total steps: {train_result.metrics['train_steps']}")
print(f"Training loss: {train_result.metrics['train_loss']:.4f}")
print(f"Training time: {train_result.metrics['train_runtime']:.0f}s")

## 8. Evaluate on Test Set

In [ ]:
print("Evaluating on test set...")
results = trainer.evaluate(tokenized_dataset["test"])

print(f"\n{'='*50}")
print("Test Set Results")
print(f"{'='*50}")
for key, value in sorted(results.items()):
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

## 9. Test Inference

In [ ]:
from transformers import pipeline

# Create summarization pipeline with fine-tuned model
summarizer = pipeline(
    "summarization",
    model=trainer.model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Test dialogues
test_dialogues = [
    """Amanda: Hey, are we meeting today?
Jerry: Sure! What time works for you?
Amanda: How about 3pm at the coffee shop?
Jerry: Perfect, see you there!
Amanda: Great, I'll bring the project reports.""",

    """Sarah: Did you finish the report?
Mike: Almost done, just need to review the numbers.
Sarah: The deadline is tomorrow at noon.
Mike: I know, I'll send it tonight.
Sarah: Great, also include the Q3 projections.
Mike: Will do. Should I CC the manager?
Sarah: Yes, please CC David.""",

    """Tom: Happy birthday! 🎂
Lisa: Thank you so much! 😊
Tom: Are you doing anything special?
Lisa: Just a small dinner with family.
Tom: That sounds lovely. Enjoy your day!"""
]

print("=" * 60)
print("INFERENCE EXAMPLES")
print("=" * 60)

for i, dialogue in enumerate(test_dialogues, 1):
    summary = summarizer(dialogue, max_length=128, min_length=10, do_sample=False)
    print(f"\n--- Example {i} ---")
    print(f"Dialogue:\n{dialogue}")
    print(f"\nSummary: {summary[0]['summary_text']}")
    print()

## 10. Save & Download Model

Save the fine-tuned model. On Kaggle, you can download from the output tab.

In [ ]:
import os

SAVE_DIR = "./bart-samsum-final"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save model and tokenizer
trainer.model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

# Show saved files
total_size = 0
for f in os.listdir(SAVE_DIR):
    size = os.path.getsize(os.path.join(SAVE_DIR, f))
    total_size += size
    print(f"  {f}: {size / 1e6:.1f} MB")

print(f"\nTotal model size: {total_size / 1e9:.2f} GB")
print(f"\nModel saved to {SAVE_DIR}")
print("\n📥 On Kaggle: Go to Output tab to download the model.")
print("📥 On Colab: Use files.download() or mount Google Drive.")

## 11. (Optional) Upload to HuggingFace Hub

Push your fine-tuned model to HuggingFace for easy deployment.

In [ ]:
# Uncomment and fill in to push to HuggingFace Hub

# from huggingface_hub import login
# login(token="hf_YOUR_TOKEN_HERE")

# HF_REPO = "your-username/bart-samsum-finetuned"
# trainer.model.push_to_hub(HF_REPO)
# tokenizer.push_to_hub(HF_REPO)
# print(f"Model pushed to https://huggingface.co/{HF_REPO}")

---

## After Training

1. **Download** the `bart-samsum-final/` folder
2. **Copy** it to your project at `artifacts/model_trainer/bart-samsum-model/`
3. **Run** the API server: `python app.py`
4. **Open** `http://localhost:8080` for the web UI

### Expected Results (3 epochs)

| Metric | Score |
|--------|-------|
| ROUGE-1 | ~0.52 |
| ROUGE-2 | ~0.28 |
| ROUGE-L | ~0.43 |
| ROUGE-Lsum | ~0.48 |